In [0]:
import math, re, os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import model_from_json
import cv2
from matplotlib import pyplot as plt
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
list_of_videofiles = glob.glob("/content/drive/My Drive/tfrecords/test3.mp4")
print(list_of_videofiles)

['/content/drive/My Drive/tfrecords/test3.mp4']


In [0]:
# !rm -r ./frames_test2/

In [0]:
parent_dir = "/content/"
for path in list_of_videofiles:
    vidcap = cv2.VideoCapture(path)
    success,image = vidcap.read()
    count = 0
    file_name  = os.path.basename(os.path.splitext(path)[0])
    if not os.path.isdir("./frames_"+ file_name):
        os.mkdir("./frames_"+ file_name)
        while success:

            cv2.imwrite("./frames_%s/%s.jpg" % (file_name,str(count).zfill(6)), image)     # save frame as JPEG file      
            success,image = vidcap.read()
            print('Read a new frame: ', success)
            count += 2

In [6]:
list_of_files = glob.glob("/content/frames_test3/*.jpg")
print(sorted(list_of_files))
print(len(list_of_files))

['/content/frames_test3/000000.jpg', '/content/frames_test3/000002.jpg', '/content/frames_test3/000004.jpg', '/content/frames_test3/000006.jpg', '/content/frames_test3/000008.jpg', '/content/frames_test3/000010.jpg', '/content/frames_test3/000012.jpg', '/content/frames_test3/000014.jpg', '/content/frames_test3/000016.jpg', '/content/frames_test3/000018.jpg', '/content/frames_test3/000020.jpg', '/content/frames_test3/000022.jpg', '/content/frames_test3/000024.jpg', '/content/frames_test3/000026.jpg', '/content/frames_test3/000028.jpg', '/content/frames_test3/000030.jpg', '/content/frames_test3/000032.jpg', '/content/frames_test3/000034.jpg', '/content/frames_test3/000036.jpg', '/content/frames_test3/000038.jpg', '/content/frames_test3/000040.jpg', '/content/frames_test3/000042.jpg', '/content/frames_test3/000044.jpg', '/content/frames_test3/000046.jpg', '/content/frames_test3/000048.jpg', '/content/frames_test3/000050.jpg', '/content/frames_test3/000052.jpg', '/content/frames_test3/0000

In [7]:
json_file = open('/content/drive/My Drive/tfrecords/model2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/tfrecords/model2.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
def pad_frame(image,pad):
    return np.pad(image, ((pad,pad),(0,0),(0,0)), 'constant')
def interpolate_frame(img):
    # img = np.concatenate((pad_frame(image1,36),pad_frame(image3,36)),axis = -1)
#     img = np.expand_dims(img, axis=0)/255.
    interpolated_image = np.zeros((6,1152,1920,3))
    img = img/255.
    # img = np.stack([img for i in range(6)],axis = 0)/255.
    # interpolated_image = np.stack([interpolated_image for i in range(6)],axis = 0)
    for row in range(9):
        for col in range(15):
            interpolated_image[:,row*128:row*128+128,col*128:col*128+128,:] = loaded_model.predict(img[:,row*128:row*128+128,col*128:col*128+128,:],batch_size = 16,use_multiprocessing = True)
    
    return interpolated_image

In [0]:
# image1 = cv2.imread('004177.jpg')
# # image2 = cv2.imread('004178.jpg')
# image3 = cv2.imread('004179.jpg')
# interpolated_image = interpolate_frame(image1,image3)


In [0]:
def create_interpolated_frames(video_frames_filepath):
  batch_of_images = []
  batch_size = 0
  interpolated_filenumber = 1
  for i in range(len(video_frames_filepath)-1):
    image1_path = video_frames_filepath[i]
    image2_path = video_frames_filepath[i+1]
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    img = np.concatenate([pad_frame(image1,36),pad_frame(image2,36)],axis = -1)
    img = np.expand_dims(img,axis = 0)
    if batch_size < 5:
      if type(batch_of_images) == list:
        batch_of_images = img 
        batch_size += 1
      else:
        batch_of_images = np.concatenate([batch_of_images,img],axis = 0)
        batch_size += 1
      continue
    else :
      batch_of_images = np.concatenate([batch_of_images,img],axis = 0)
      batch_size += 1
      interpolated_images = interpolate_frame(batch_of_images)
      for j in range(6):
        image_name = './frames_test3/'+str(interpolated_filenumber).zfill(6)+'.jpg'
        cv2.imwrite(image_name,interpolated_images[j,:,:,:]*255.)
        interpolated_filenumber += 2
      batch_size = 0
      batch_of_images = []
      print("yup")


In [10]:
print(list_of_files)

['/content/frames_test3/000568.jpg', '/content/frames_test3/000778.jpg', '/content/frames_test3/000746.jpg', '/content/frames_test3/001028.jpg', '/content/frames_test3/000890.jpg', '/content/frames_test3/000132.jpg', '/content/frames_test3/000490.jpg', '/content/frames_test3/000940.jpg', '/content/frames_test3/000820.jpg', '/content/frames_test3/000124.jpg', '/content/frames_test3/000880.jpg', '/content/frames_test3/000822.jpg', '/content/frames_test3/000814.jpg', '/content/frames_test3/000014.jpg', '/content/frames_test3/000922.jpg', '/content/frames_test3/000212.jpg', '/content/frames_test3/000858.jpg', '/content/frames_test3/000010.jpg', '/content/frames_test3/000198.jpg', '/content/frames_test3/000088.jpg', '/content/frames_test3/000216.jpg', '/content/frames_test3/000964.jpg', '/content/frames_test3/000176.jpg', '/content/frames_test3/000564.jpg', '/content/frames_test3/000426.jpg', '/content/frames_test3/001032.jpg', '/content/frames_test3/000768.jpg', '/content/frames_test3/0001

In [11]:
create_interpolated_frames(sorted(list_of_files))

yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup
yup


In [0]:
def cut_extra(video_frames_list):
  x = 1
  for frame in video_frames_list:
    file_name  = os.path.basename(os.path.splitext(frame)[0])
    current_file_to_cut = str(x).zfill(6)
    if(current_file_to_cut==file_name):
      img = cv2.imread(frame)
      img = img[36:1080+36,:,:]
      cv2.imwrite(frame,img)
      x += 2

In [13]:
list_of_files = sorted(glob.glob("/content/frames_test3/*.jpg"))
print(list_of_files)

['/content/frames_test3/000000.jpg', '/content/frames_test3/000001.jpg', '/content/frames_test3/000002.jpg', '/content/frames_test3/000003.jpg', '/content/frames_test3/000004.jpg', '/content/frames_test3/000005.jpg', '/content/frames_test3/000006.jpg', '/content/frames_test3/000007.jpg', '/content/frames_test3/000008.jpg', '/content/frames_test3/000009.jpg', '/content/frames_test3/000010.jpg', '/content/frames_test3/000011.jpg', '/content/frames_test3/000012.jpg', '/content/frames_test3/000013.jpg', '/content/frames_test3/000014.jpg', '/content/frames_test3/000015.jpg', '/content/frames_test3/000016.jpg', '/content/frames_test3/000017.jpg', '/content/frames_test3/000018.jpg', '/content/frames_test3/000019.jpg', '/content/frames_test3/000020.jpg', '/content/frames_test3/000021.jpg', '/content/frames_test3/000022.jpg', '/content/frames_test3/000023.jpg', '/content/frames_test3/000024.jpg', '/content/frames_test3/000025.jpg', '/content/frames_test3/000026.jpg', '/content/frames_test3/0000

In [0]:
cut_extra(sorted(list_of_files))

In [0]:
# !zip -r frames_test.zip frames_test

In [0]:
def convert_frames_to_video(list_of_files,pathOut,fps):

    frame_array = []
    for i in range(len(list_of_files)):
        # filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(list_of_files[i])
        height, width, layers = img.shape
        size = (width,height)
        print(list_of_files[i])
        #inserting the frames into an image array
        frame_array.append(img)

    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [0]:
convert_frames_to_video(list_of_files,'./test2_interpolated.avi',60)

In [0]:
cv2.imwrite('interpolated_frame.jpg',interpolated_image[1]*255.)

True